In [1]:
delete this line
i
import pandas as pd
import os 
import h3 as h3
import matplotlib.pyplot as plt
import plotly
import folium
import webbrowser
from folium import Map

from IPython.core.display import display, HTML
import json
from pandas.io.json import json_normalize
import numpy as np

import statistics

from geojson.feature import *
# from area import area

import copy
from folium import Map, Marker, GeoJson
from folium.plugins import MarkerCluster
import branca.colormap as cm
from branca.colormap import linear

import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image, display
from IPython.utils.text import columnize
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

SyntaxError: invalid syntax (<ipython-input-1-102b7d9277ce>, line 1)

In [ ]:
visits_count = pd.read_csv('~/Downloads/tt_visitsByWard_2018.csv')
# visits_count = pd.read_csv('~/Downloads/visits_every_ward.csv')
wards_boundaries = pd.read_csv('~/Downloads/Wards__December_2017__Boundaries_in_the_UK.csv')

In [ ]:
# s1 = pd.merge(visits_count, wards_boundaries, how='inner', on=['ward_code'])
# # s1.dropna(inplace=True)   # Drop any rows with NA
# s1.shape


In [ ]:
# s1.head()

In [ ]:
s1.to_csv(path_or_buf = ('~/Downloads/s1.csv'), index=False)  # export the dataframe to a csv file

In [ ]:
df = pd.read_csv('~/Downloads/visits_every_ward.csv')
ori = pd.read_csv('~/Downloads/visits_every_ward.csv')    
# print(df.head())


In [2]:
# all_wards = list(wards_boundaries['ward_code'])
# # len(all_wards)

# fb_wards = list(visits_count['ward_code'])
# # len(fb_wards) #   ---> 7397

# duplicate_s1 = s1.copy()
# # duplicate_s1.head()

# unknown_wards = []
# for i in fb_wards:
#     if i not in all_wards:
#         unknown_wards.append(i)

In [3]:
# len(unknown_wards)


In [4]:
# s2 = pd.merge(wards_boundaries, duplicate_s1, how='outer', on=['ward_code'])
# s2.head()
# s2.to_csv('~/downloads/visits_every_ward.csv')

In [5]:
def counts_by_hexagon(df, resolution):

    '''Use h3.geo_to_h3 to index each data point into the spatial index of the specified resolution.
      Use h3.h3_to_geo_boundary to obtain the geometries of these hexagons'''

    df = df[["lat","lng"]]

    df["hex_id"] = df.apply(lambda row: h3.geo_to_h3(row["lat"], row["lng"], resolution), axis = 1)

    df_aggreg = df
    df_aggreg['n_of_visits'] = ori['n_of_visits']

    df_aggreg.columns = ["lat", "lng", "hex_id", "n_of_visits"]

    df_aggreg["geometry"] =  df_aggreg.hex_id.apply(lambda x: 
                                                           {    "type" : "Polygon",
                                                                 "coordinates": 
                                                                [h3.h3_to_geo_boundary(x,geo_json=True)]
                                                            }
                                                        )

    return df_aggreg


def hexagons_dataframe_to_geojson(df_hex, file_output = None):
    """
    Produce the GeoJSON for a dataframe that has a geometry column in geojson 
    format already, along with the columns hex_id and value

    Ex counts_by_hexagon(data)
    """    
    list_features = []

    for i,row in df_hex.iterrows():
        feature = Feature(geometry = row["geometry"] , id=row["hex_id"], properties = {"n_of_visits" : row["n_of_visits"]})
        list_features.append(feature)

    feat_collection = FeatureCollection(list_features)

    geojson_result = json.dumps(feat_collection)

    #optionally write to file
    if file_output is not None:
        with open(file_output,"w") as f:
            json.dump(feat_collection,f)

    return geojson_result

def choropleth_map(df_aggreg, border_color = 'black', fill_opacity = 0.7, initial_map = None, with_legend = False,
               kind = "linear"):

    """
    Creates choropleth maps given the aggregated data.
    """    
    #colormap
    min_value = df_aggreg["n_of_visits"].min()
    max_value = df_aggreg["n_of_visits"].max()
    m = round ((min_value + max_value ) / 2 , 0)

    #take resolution from the first row
    res = h3.h3_get_resolution(df_aggreg.loc[0,'hex_id'])

    if initial_map is None:
        initial_map = Map(location= [51.53, 0.11], zoom_start=7, tiles="cartodbpositron", 
                attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>' 
            )


    #the colormap 
    #color names accepted https://github.com/python-visualization/branca/blob/master/branca/_cnames.json
    if kind == "linear":
        custom_cm = cm.LinearColormap(colors=['green','lightyellow','yellow','red'], index=[0,50,150,2000],vmin=min_value, vmax=max_value)
    elif kind == "outlier":
        #for outliers, values would be -11,0,1
        custom_cm = cm.LinearColormap(['blue','white','red'], vmin=min_value, vmax=max_value)
    elif kind == "filled_nulls":
        custom_cm = cm.LinearColormap(['sienna','green','yellow','red'], 
                                      index=[0,min_value,m,max_value],vmin=min_value,vmax=max_value)


    #create geojson data from dataframe
    geojson_data = hexagons_dataframe_to_geojson(df_hex = df_aggreg)

    #plot on map
    name_layer = "Choropleth " + str(res)
    if kind != "linear":
        name_layer = name_layer + kind

    GeoJson(
        geojson_data,
        style_function=lambda feature: {
            'fillColor': custom_cm(feature['properties']['n_of_visits']),
            'color': border_color,
            'weight': 1,
            'fillOpacity': fill_opacity 
        }, 
        name = name_layer
    ).add_to(initial_map)
    #add legend (not recommended if multiple layers)
    if with_legend == True:
        custom_cm.add_to(initial_map)



    return initial_map

def plot_scatter(df, metric_col, x='lng', y='lat', marker='.', alpha=1, figsize=(16,12), colormap='viridis'):
    """
    Scatter plot function for h3 indexed objects
    """    
    df.plot.scatter(x=x, y=y, c=metric_col, title=metric_col
                    , edgecolors='none', colormap=colormap, marker=marker, alpha=alpha, figsize=figsize);
    plt.xticks([], []); plt.yticks([], [])


In [6]:
# Counts how many points are within the hex
myres = 6 # use 6
df_aggreg = counts_by_hexagon(df = df, resolution = myres)      #Use 4 to tile the whole of uk in an aesthetically pleasing way
df_aggreg.sort_values(by = "n_of_visits", ascending = False, inplace = True)

# Creates a map using Folium
hexmap = choropleth_map(df_aggreg = df_aggreg, with_legend = True)
hexmap.save('downloads')
from folium import plugins
# add full screen button to map
plugins.Fullscreen(position='topright').add_to(hexmap)
draw = plugins.Draw(export=True)
hexmap



NameError: name 'df' is not defined

In [ ]:
df_aggreg

In [ ]:
unique_hex = set(df_aggreg["hex_id"])

In [ ]:
len(unique_hex)